In [1]:
# Table 4 Incidents by year
import pandas as pd
import plotly.graph_objects as go


# Load the data
df = pd.read_csv("../data/data-security-incidents-trends-q1-2019-to-q4-2024.csv") 

# Create unique incident identifier
df['Incident_ID'] = df['BI Reference'] + '_' + df['Year'].astype(str) + '_' + df['Quarter']

# Calculate yearly totals and percentages
yearly_data = []
total_all_years = df['Incident_ID'].nunique()


for year in sorted(df['Year'].unique()):
    year_data = df[df['Year'] == year]
    total_incidents = year_data['Incident_ID'].nunique()
    relative_frequency = total_incidents/total_all_years
    yearly_data.append([year, total_incidents, relative_frequency])


# Create table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['Year', 'Total Incidents', 'Percentage'],
        font=dict(size=14, color='white'),
        fill_color='darkblue',
        align='center'
    ),
    cells=dict(
        values=[
            [year for year, _, _ in yearly_data],
            [total for _, total, _ in yearly_data],
            [f"{round(relative_frequency*100, 1)}%" for _, _, relative_frequency in yearly_data]
        ],
        font=dict(size=13),
        align='center',
        format=[None, ',',None], 
        height=30
    )
)])

# Update layout
fig.update_layout(
    title={
        'text': 'Yearly Frequency of Incidents from Q1 2019 to Q2 2024</sub>',
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=14)
    },
    width=600,
    height=400
)

# Show table
fig.show()



In [2]:
# Figure 1: Incidents per year bar chart
import plotly.graph_objects as go 


# Extract just the year and total values
years = [item[0] for item in yearly_data]
totals = [item[1] for item in yearly_data]
perc = [round(item[2]*100, 1) for item in yearly_data]
total_sum = sum(totals)

# Create bar graph
fig = go.Figure(data=[
    go.Bar(
        x=years, 
        y=totals,
        text=[f'{p}%' for p in perc],
        textposition='outside',
        marker_color='rgb(49, 130, 189)'
    )
])

# Update layout
fig.update_layout(
    title={
        'text': 'Yearly Frequency of Incidents from Q1 2019 to Q2 2024',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=14)
    },
    xaxis_title="Year",
    yaxis_title="Number of Incidents",
    width=800,
    height=500,
    showlegend=False,
    plot_bgcolor='white',
    yaxis=dict(
        gridcolor='lightgrey',
        gridwidth=1
    )
)

# Show figure
fig.show()


In [3]:
# Figure 2: Incidents per quarter line chart
import pandas as pd
import plotly.express as px

# Read the CSV file
df = pd.read_csv('../data/data-security-incidents-trends-q1-2019-to-q4-2024.csv')

# Replace Qtr with Q for clearer visualisation
df['Quarter'] = df['Quarter'].str.replace('Qtr ', 'Q')

# Create unique Incident ID
df['Incident_ID'] = df['BI Reference'] + '_' + df['Year'].astype(str) + '_' + df['Quarter']

# Create a datetime label for Year + Quarter
quarter_to_month = {'Q1': '01', 'Q2': '04', 'Q3': '07', 'Q4': '10'}
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Quarter'].map(quarter_to_month) + '-01')

# Drop duplicate Incident_IDs
unique_incidents = df.drop_duplicates(subset='Incident_ID')

# Group by Date and count distinct incidents
grouped = unique_incidents.groupby('Date').size().reset_index(name='Unique Incidents')

# Plot time series
fig = px.line(grouped, x='Date', y='Unique Incidents', markers=True,
              title='Data Security Incidents by Year and Quarter',
              labels={'Date': 'Quarter', 'Unique Incidents': 'Incident Count'})

fig.update_layout(
    xaxis_title='Year and Quarter',
    yaxis_title='Number of Incidents',
    xaxis=dict(dtick='M3', tickformat='%Y Q%q')
)

fig.show()

In [4]:
# Figure 3: Incidents per category bar chart
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read the CSV file
df = pd.read_csv('../data/data-security-incidents-trends-q1-2019-to-q4-2024.csv')

# Create unique incident identifier
df['Incident_ID'] = df['BI Reference'] + '_' + df['Year'].astype(str) + '_' + df['Quarter']

# Calculate yearly totals by Incident Category
yearly_by_category = df.groupby(['Year', 'Incident Category'])['Incident_ID'].nunique().unstack(fill_value=0)

# Create grouped bar chart
fig = go.Figure()

# Add a bar for each incident category
for category in yearly_by_category.columns:
    fig.add_trace(
        go.Bar(
            name=category,
            x=yearly_by_category.index,
            y=yearly_by_category[category],
            text=[f"{val:,}" for val in yearly_by_category[category]],
            textposition='outside'
        )
    )

# Update layout
fig.update_layout(
    title={
        'text': 'Yearly Distribution by Incident Category',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=14)
    },
    xaxis_title="Year",
    yaxis_title="Number of Incidents",
    barmode='group',  
    width=800,  
    height=500,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=1.2
    ),
    plot_bgcolor='white',
    yaxis=dict(
        gridcolor='lightgrey',
        gridwidth=1
    )
)

fig.show()

In [5]:
# Figure 4: Top Non-Cyber Incident Types 
import pandas as pd
import plotly.express as px

# Read the CSV file
df = pd.read_csv('../data/data-security-incidents-trends-q1-2019-to-q4-2024.csv')

# Create unique incident ID 
df['Incident_ID'] = df['BI Reference'] + '_' + df['Year'].astype(str) + '_' + df['Quarter']

# Filter for non-cyber incidents
non_cyber_df = df[df["Incident Category"] == "Non Cyber"]

# Count number of unique incidents per type
incident_counts = non_cyber_df["Incident Type"].value_counts().reset_index()
incident_counts.columns = ["Incident Type", "Count"]

# Select top 10 types 
incident_counts = incident_counts.head(10)

# Create the pie chart
fig = px.pie(incident_counts,
             names="Incident Type",
             values="Count",
             title="Top Non-Cyber Incident Types",
             hole=0.3)

# Set figure size and layout
fig.update_layout(
    width=700,   
    height=400,  
    legend=dict(
        font=dict(size=10),
        orientation="v",       
        x=1,                   
        y=0.5,
        xanchor="left"
    ),
    title_font=dict(size=14)
)
fig.update_traces(textfont_size=9) 

# Show the chart
fig.show()


In [6]:
# Figure 5: Top Cyber Incident Types 
import pandas as pd
import plotly.express as px

# Read the CSV file
df = pd.read_csv('../data/data-security-incidents-trends-q1-2019-to-q4-2024.csv')

# Create unique incident ID 
df['Incident_ID'] = df['BI Reference'] + '_' + df['Year'].astype(str) + '_' + df['Quarter']

# Filter for non-cyber incidents
non_cyber_df = df[df["Incident Category"] == "Cyber"]

# Count number of unique incidents per type
incident_counts = non_cyber_df["Incident Type"].value_counts().reset_index()
incident_counts.columns = ["Incident Type", "Count"]

# Select top 10 types 
incident_counts = incident_counts.head(10)

# Create the pie chart
fig = px.pie(incident_counts,
             names="Incident Type",
             values="Count",
             title="Top Cyber Incident Types",
             hole=0.3)

# Set figure size and layout
fig.update_layout(
    width=600,   
    height=400,  
    legend=dict(
        font=dict(size=10),
        orientation="v",       
        x=1,                   
        y=0.5,
        xanchor="left"
    ),
    title_font=dict(size=14)
)
fig.update_traces(textfont_size=9) 

# Show the chart
fig.show()

In [7]:
# Figure 6: Distribution of Incidents by Sector 
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read the CSV file
df = pd.read_csv('../data/data-security-incidents-trends-q1-2019-to-q4-2024.csv')

# Create unique incident identifier
df['Incident_ID'] = df['BI Reference'] + '_' + df['Year'].astype(str) + '_' + df['Quarter']

# Calculate sector totals
sector_counts = df.groupby('Sector')['Incident_ID'].nunique().sort_values(ascending=True)
total_incidents = sector_counts.sum()
sector_percentages = (sector_counts / total_incidents * 100).round(1)

# Create horizontal bar chart
fig = go.Figure(data=[
    go.Bar(
        y=sector_counts.index,
        x=sector_counts.values,
        orientation='h',
        text=[f"{pct}%" for pct in sector_percentages],
        textposition='outside',
        marker_color='rgb(49, 130, 189)'
    )
])

# Update layout
fig.update_layout(
    title={
        'text': 'Distribution of Incidents by Sector',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=16)
    },
    xaxis_title="Number of Incidents",
    yaxis_title=None,  
    width=900,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(
        gridcolor='lightgrey',
        gridwidth=1
    )
)

fig.show()

In [8]:
# Figure 5: Top Cyber Incident Types 
import pandas as pd
import plotly.express as px

# Read the CSV file
df = pd.read_csv('../data/data-security-incidents-trends-q1-2019-to-q4-2024.csv')

# Create unique incident ID 
df['Incident_ID'] = df['BI Reference'] + '_' + df['Year'].astype(str) + '_' + df['Quarter']

# Filter for non-cyber incidents
non_cyber_df = df[df["Sector"] == "Finance, insurance and credit"]

# Count number of unique incidents per type
incident_counts = non_cyber_df["Incident Type"].value_counts().reset_index()
incident_counts.columns = ["Incident Type", "Count"]

# Select top 10 types 
incident_counts = incident_counts.head(10)

# Create the pie chart
fig = px.pie(incident_counts,
             names="Incident Type",
             values="Count",
             title="Top Cyber Incident Types",
             hole=0.3)

# Set figure size and layout
fig.update_layout(
    width=600,   
    height=400,  
    legend=dict(
        font=dict(size=10),
        orientation="v",       
        x=1,                   
        y=0.5,
        xanchor="left"
    ),
    title_font=dict(size=14)
)
fig.update_traces(textfont_size=9) 

# Show the chart
fig.show()